In [57]:
import os
from dotenv import load_dotenv

from oci.auth.signers import InstancePrincipalsSecurityTokenSigner
from oci.generative_ai.generative_ai_client import GenerativeAiClient
# from oci.generative_ai.models import 
from oci.generative_ai_inference.generative_ai_inference_client import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import (
    ChatDetails,
    OnDemandServingMode,
    CohereChatRequest, 
    CohereTool, 
    CohereParameterDefinition,
    CohereToolCall,
    CohereToolResult
)

In [58]:
load_dotenv()
compartment_id = os.getenv("COMPARTMENT_ID")
endpoint = os.getenv("MANAGEMENT_ENDPOINT")
i_endpoint = os.getenv("INFERENCE_ENDPOINT")
signer = InstancePrincipalsSecurityTokenSigner()
generative_ai_client = GenerativeAiClient(config = {}, signer = signer, service_endpoint = endpoint)
generative_ai_inference_client = GenerativeAiInferenceClient(config = {}, signer = signer, service_endpoint = i_endpoint)

In [59]:
list_models_response = generative_ai_client.list_models(
    compartment_id=compartment_id,
)

print(list_models_response.data)

{
  "items": [
    {
      "base_model_id": null,
      "capabilities": [
        "CHAT"
      ],
      "compartment_id": null,
      "defined_tags": {},
      "display_name": "cohere.command-r-16k",
      "fine_tune_details": null,
      "freeform_tags": {},
      "id": "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq",
      "is_long_term_supported": true,
      "lifecycle_details": "Creating Base Model",
      "lifecycle_state": "ACTIVE",
      "model_metrics": null,
      "system_tags": {},
      "time_created": "2024-05-30T22:43:42.344000+00:00",
      "time_deprecated": null,
      "type": "BASE",
      "vendor": "cohere",
      "version": "1.2"
    },
    {
      "base_model_id": null,
      "capabilities": [
        "CHAT"
      ],
      "compartment_id": null,
      "defined_tags": {},
      "display_name": "meta.llama-3-70b-instruct",
      "fine_tune_details": null,
      "freeform_tags": {},
      "id": "ocid1.generative

In [60]:
response = generative_ai_inference_client.chat(
    chat_details=ChatDetails(
        compartment_id=compartment_id,
        serving_mode=OnDemandServingMode(
            model_id="cohere.command-r-16k"
        ),
        chat_request=CohereChatRequest(
            message="日本で一番高い山はなんですか？",
            max_tokens=200,
            
        )
    )
)

print(response.data)

{
  "chat_response": {
    "api_format": "COHERE",
    "chat_history": [
      {
        "message": "\u65e5\u672c\u3067\u4e00\u756a\u9ad8\u3044\u5c71\u306f\u306a\u3093\u3067\u3059\u304b\uff1f",
        "role": "USER"
      },
      {
        "message": "\u5bcc\u58eb\u5c71",
        "role": "CHATBOT",
        "tool_calls": null
      }
    ],
    "citations": null,
    "documents": null,
    "error_message": null,
    "finish_reason": "COMPLETE",
    "is_search_required": null,
    "prompt": null,
    "search_queries": null,
    "text": "\u5bcc\u58eb\u5c71",
    "tool_calls": null
  },
  "model_id": "cohere.command-r-16k",
  "model_version": "1.2"
}


In [62]:
tools = [
    CohereTool(
        name = "query_daily_sales_report",
        description = "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        parameter_definitions = {
            "day": CohereParameterDefinition(
                description = "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                type = "str",
                is_required = True
            )
        }
    ),
    CohereTool(
        name = "query_product_catalog",
        description = "Connects to a a product catalog with information about all the products being sold, including categories, prices, and stock levels.",
        parameter_definitions = {
            "category": CohereParameterDefinition(
                description = "Retrieves product information data for all products in this category.",
                type = "str",
                is_required = True
            )
        }
    )
]

preamble_override_en = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

response = response = generative_ai_inference_client.chat(
    chat_details=ChatDetails(
        compartment_id=compartment_id,
        serving_mode=OnDemandServingMode(
            model_id="cohere.command-r-16k"
        ),
        chat_request=CohereChatRequest(
            message="Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?",
            max_tokens=200,
            tools=tools_en,
            preamble_override=preamble_override_en
        )
    )
)

print(response.data)

{
  "chat_response": {
    "api_format": "COHERE",
    "chat_history": [
      {
        "message": "Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?",
        "role": "USER"
      },
      {
        "message": "I will first search for the sales summary of 29th September 2023 and then search for product details in the Electronics category.",
        "role": "CHATBOT",
        "tool_calls": [
          {
            "name": "query_daily_sales_report",
            "parameters": {
              "day": "2023-09-29"
            }
          }
        ]
      }
    ],
    "citations": null,
    "documents": null,
    "error_message": null,
    "finish_reason": "COMPLETE",
    "is_search_required": null,
    "prompt": null,
    "search_queries": null,
    "text": "I will first search for the sales summary of 29th September 2023 and then search for product details i

In [64]:
print("The model recommends doing the following tool calls:")
print("\n".join(str(tool_call) for tool_call in response.data.chat_response.tool_calls))

The model recommends doing the following tool calls:
{
  "name": "query_daily_sales_report",
  "parameters": {
    "day": "2023-09-29"
  }
}


In [65]:
# Mock database containing daily sales reports
sales_database = {
    '2023-09-28': {
        'total_sales_amount': 5000,
        'total_units_sold': 100,
    },
    '2023-09-29': {
        'total_sales_amount': 10000,
        'total_units_sold': 250,
    },
    '2023-09-30': {
        'total_sales_amount': 8000,
        'total_units_sold': 200,
    }
}

# Mock product catalog
product_catalog = {
    'Electronics': [
        {'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20},
        {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15},
        {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25},
    ],
    'Clothing': [
        {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 20, 'stock_level': 100},
        {'product_id': 'C1002', 'name': 'Jeans', 'price': 50, 'stock_level': 80},
        {'product_id': 'C1003', 'name': 'Jacket', 'price': 100, 'stock_level': 40},
    ]
}

def query_daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    report = sales_database.get(day, {})
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}


def query_product_catalog(category: str) -> dict:
    """
    Function to retrieve products for the given category
    """
    products = product_catalog.get(category, [])
    return {
        'category': category,
        'products': products
    }


functions_map = {
    "query_daily_sales_report": query_daily_sales_report,
    "query_product_catalog": query_product_catalog
}


In [67]:
tool_results = []
for tool_call in response.data.chat_response.tool_calls:
    output = functions_map[tool_call.name](**tool_call.parameters)
    outputs = [output]
    tool_results.append(
        CohereToolResult(
            call=CohereToolCall(
                name=tool_call.name,
                parameters=tool_call.parameters
            ),
            outputs=outputs
        )
    )

print("Tool results that will be fed back to the model in step 4:")
print(tool_results)

Tool results that will be fed back to the model in step 4:
[{
  "call": {
    "name": "query_daily_sales_report",
    "parameters": {
      "day": "2023-09-29"
    }
  },
  "outputs": [
    {
      "date": "2023-09-29",
      "summary": "Total Sales Amount: 10000, Total Units Sold: 250"
    }
  ]
}]


In [69]:
response = generative_ai_inference_client.chat(
    chat_details=ChatDetails(
        compartment_id=compartment_id,
        serving_mode=OnDemandServingMode(
            model_id="cohere.command-r-16k"
        ),
        chat_request=CohereChatRequest(
            message="Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?",
            max_tokens=200,
            is_force_single_step=True,
            tools=tools,
            tool_results=tool_results,
            preamble_override=preamble_override_en
        )
    )
)

print(response.data)

{
  "chat_response": {
    "api_format": "COHERE",
    "chat_history": [
      {
        "message": "Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?",
        "role": "USER"
      },
      {
        "message": "Here's a summary of sales for 29th September 2023:\n\n- Total Sales Amount: \u00a310000\n- Total Units Sold: 250\n\nI'm afraid I can't",
        "role": "CHATBOT",
        "tool_calls": null
      }
    ],
    "citations": [
      {
        "document_ids": [
          "query_daily_sales_report:0:0"
        ],
        "end": 80,
        "start": 54,
        "text": "Total Sales Amount: \u00a310000"
      },
      {
        "document_ids": [
          "query_daily_sales_report:0:0"
        ],
        "end": 104,
        "start": 83,
        "text": "Total Units Sold: 250"
      }
    ],
    "documents": [
      {
        "date": "2023-09-29",
        